# Turkish ULMFiT from scratch

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from venkateshreddypala import *
from venkateshreddypala.text import *

In [0]:
bs=128
torch.cuda.set_device(2)
data_path = Config.data_path()

lang = 'tr'
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)

In [0]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
lm_fns = [mdl_path/f'{lang}_wt', mdl_path/f'{lang}_wt_vocab']

## Turkish wikipedia model

In [0]:
from nlputils import split_wiki,get_wiki

get_wiki(path,lang)
!head -n4 {path}/{name}

In [0]:
dest = split_wiki(path,lang)

Turkish is an [Agglutinative_language](https://en.wikipedia.org/wiki/Agglutinative_language) so it needs special care!

In [0]:
data = (TextList.from_folder(dest, processor=[OpenFileProcessor(), SPProcessor()])
        .split_by_rand_pct(0.1, seed=42)
        .label_for_lm()
        .databunch(bs=bs, num_workers=1))

data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

In [0]:
data = load_data(dest, f'{lang}_databunch', bs=bs)

In [0]:
data.show_batch()

In [0]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.1, wd=0.1, pretrained=False).to_fp16()

In [0]:
lr = 3e-3
lr *= bs/48  # Scale learning rate by batch size

In [0]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,4.337346,4.472464,0.297502,10:04
1,4.145160,4.359449,0.304195,10:29
2,4.199796,4.386935,0.301542,10:29
3,4.132432,4.329662,0.305723,10:30
4,4.048324,4.241048,0.313223,10:32
5,3.943493,4.118834,0.323640,10:30
6,3.857112,3.969563,0.337475,10:30
7,3.695721,3.799551,0.354562,10:30
8,3.602813,3.652005,0.371855,10:31
9,3.519019,3.596706,0.379076,10:31


In [0]:
learn.to_fp32().save(lm_fns[0], with_opt=False)
learn.data.vocab.save(lm_fns[1].with_suffix('.pkl'))

## Turkish sentiment analysis

https://www.win.tue.nl/~mpechen/projects/smm/

### Language model

In [0]:
path_clas = path/'movies'
path_clas.ls()

[PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_polarity.pos'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_clas_databunch'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/models'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_polarity.neg'),
 PosixPath('/home/sgugger/.fastai/data/trwiki/movies/tr_data_lm')]

In [0]:
pos = (path_clas/'tr_polarity.pos').open(encoding='iso-8859-9').readlines()
pos_df = pd.DataFrame({'text':pos})
pos_df['pos'] = 1
pos_df.head()

,text,pos
0,gerçekten harika bir yapim birçok kez izledim ...,1
1,her izledigimde hayranlik duydugum gerçek klas...,1
2,gerçekten tarihi savas filmleri arasinda tarti...,1
3,aldigi ödülleri sonuna dek hak eden muhtesem b...,1
4,özgürlük denilince aklima gelen ilk film.bir b...,1


In [0]:
neg = (path_clas/'tr_polarity.neg').open(encoding='iso-8859-9').readlines()
neg_df = pd.DataFrame({'text':neg})
neg_df['pos'] = 0
neg_df.head()

,text,pos
0,giseye oynayan bir film.mel gibson'in oyunculu...,0
1,bircok yonden sahip olduklari zayifliklari pop...,0
2,"1995 ten bu yana bu tür filmler artti , o zama...",0
3,mel gibson tam bir ingiliz düsmani her filmind...,0
4,milliyetçi bir film tavsiye etmiyorum.... \n,0


In [0]:
df = pd.concat([pos_df,neg_df], sort=False)

In [0]:
data_lm = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_for_lm()           
    .databunch(bs=bs, num_workers=1))

data_lm.save(f'{lang}_clas_databunch')

In [0]:
data_lm = load_data(path_clas, f'{lang}_clas_databunch', bs=bs)

In [0]:
data_lm.show_batch()

idx,text
0,"▁bile ▁sinema ▁olgusu nun ▁en ▁üst ▁noktalar in dan . . ▁xxbos ▁gerçekten ▁tarihi ▁sava s ▁filmleri ▁ara si nda ▁tar tis ma siz ▁en ▁iyi si ▁ , ▁12 ▁ yi l ▁boyunca ▁ac aba ▁ikincisi ▁çek ir imi ▁diye ▁bekledi gi m ▁bir ▁film ▁ , bel ki ▁william ▁wallace ▁baba sinin ▁ölümünden ▁sonra ▁amca si ▁yani na ▁al m isti ▁onu ▁ ye tis tir m isti"
1,ka dra j lar i yla ▁konusu yla ▁is le ni siyle ▁insana ▁ iste ▁film ▁böyle ▁çekilir ▁de dir ten ▁kusur suz ▁bir ▁film . ▁xxbos ▁böyle ▁güzel ▁bir ▁yap it ▁olamaz ▁filmde ▁her ▁sey ▁var ▁insani ▁dünya dan ▁ali p ▁gö tür üyor ▁bask a ▁diyar lara ▁film ▁bitti kten ▁sonra ▁epey ▁süre ▁geçmesi ▁gerekiyor ▁tekrar ▁dünya ▁ya ▁dönmek ▁için ▁dikkat ! . ▁xxbos ▁inan ir mi siniz
2,▁o ▁kadar ▁etki len mis tim ▁ki . . . özellikle ▁özgürlük ▁ ug ru na ▁sava san ▁william ▁wallace  in ▁is ken ce ▁edilerek ▁idam ▁edilmesi . . . ve ▁sonunda ▁özgürlük ▁diye ▁hay kir isi . . . ha lan ▁unut ami yorum ▁xxrep ▁4 ▁ . ▁xxbos ▁ilk ▁bu ▁filmi ▁sinema da ▁izledi m ▁ve ▁insan in ▁inan di ktan ▁sonra ▁ ne leri ▁yap a
3,▁için ▁ister ▁istemez ▁oraya ▁gö tür üyor ▁filmin ▁uzun lu gun a ▁al dan ip ta ▁filmi ▁izlemek ten ▁vazgeçme yin ▁xxrep ▁4 ▁ . ▁xxbos ▁harika ▁bir ▁film di ▁xxrep ▁5 ▁ . ▁xxbos ▁mükemmel ▁ötesi . . ▁ . ▁xxbos ▁hiç ▁ a bart mi yorum ▁hayat im da ▁izledi gi m ▁en ▁iyi ▁film lerden ▁biri ▁diye bilir im . tam ▁bir ▁bas yap it ▁nite ligi
4,les iyor . ▁herkes ▁izleme li . . . ▁xxbos ▁tarantino nun ▁en ▁iyi ▁filmlerinden ▁biri . ▁diyalog lar ▁çok ▁iyi . ▁kesinlikle ▁izlenmesi ▁gereken ▁bir ▁film . . ▁xxbos ▁tarantino nun ▁bu ▁filmi ▁kendini ▁belli ▁etti r iyor . hat ta ▁ben ce ▁tarantino nun ▁en ▁iyi ▁filmidir . kendi ne ▁has ▁anlat imi ▁ile ▁bu ▁film ▁hak ka ten ▁sinema ▁sever lerin ▁izleme si ▁gereken ▁bir ▁film .


In [0]:
learn_lm = language_model_learner(data_lm, AWD_LSTM, pretrained_fnames=lm_fns, drop_mult=1.0, wd=0.1)

In [0]:
lr = 1e-3
lr *= bs/48

In [0]:
learn_lm.fit_one_cycle(1, lr*10, moms=(0.8,0.7))

epoch,train_loss,valid_loss,time
0,4.762205,4.132825,00:07


In [0]:
learn_lm.unfreeze()
learn_lm.fit_one_cycle(5, slice(lr/10,lr*10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,time
0,4.136730,3.976010,00:08
1,3.991336,3.882426,00:08
2,3.768218,3.812365,00:08
3,3.526307,3.795765,00:08
4,3.310204,3.815228,00:08


In [0]:
learn_lm.save(f'{lang}fine_tuned')
learn_lm.save_encoder(f'{lang}fine_tuned_enc')

### Classifier

In [0]:
data_clas = (TextList.from_df(df, path_clas, cols='text', processor=SPProcessor.load(dest))
    .split_by_rand_pct(0.1, seed=42)
    .label_from_df(cols='pos')
    .databunch(bs=bs, num_workers=1))

In [0]:
learn_c = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False, wd=0.1).to_fp16()
learn_c.load_encoder(f'{lang}fine_tuned_enc')
learn_c.freeze()

In [0]:
lr=2e-2
lr *= bs/48

In [0]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.456195,0.441739,0.817073,00:02
1,0.416383,0.359390,0.841463,00:02


In [0]:
learn_c.fit_one_cycle(2, lr, moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.413642,0.381773,0.828330,00:02
1,0.407912,0.358319,0.846154,00:02


In [0]:
learn_c.freeze_to(-2)
learn_c.fit_one_cycle(2, slice(lr/(2.6**4),lr), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.358337,0.310041,0.869606,00:02
1,0.279921,0.276278,0.892120,00:02


In [0]:
learn_c.freeze_to(-3)
learn_c.fit_one_cycle(2, slice(lr/2/(2.6**4),lr/2), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.258410,0.261940,0.899625,00:03
1,0.183993,0.267610,0.905253,00:02


In [0]:
learn_c.unfreeze()
learn_c.fit_one_cycle(4, slice(lr/10/(2.6**4),lr/10), moms=(0.8,0.7))

epoch,train_loss,valid_loss,accuracy,time
0,0.115543,0.290134,0.902439,00:03
1,0.107257,0.297127,0.911820,00:03


Accuracy in Gezici (2018), *Sentiment Analysis in Turkish* is: `75.16%`.

In [0]:
learn_c.save(f'{lang}clas')